This code accompanies the paper submitted on July 1, 2019: Theory of time-resolved optical conductivity of
superconductors: comparing two methods for its evaluation.

Here we import the important packages and define the useful functions for this analysis.

In [4]:
%pylab inline


from scipy import interpolate
import matplotlib.ticker as mtick
from matplotlib.ticker import ScalarFormatter
from matplotlib.patches import Rectangle
import matplotlib.cm as mplcm
from matplotlib.cm import coolwarm
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from matplotlib.animation import FuncAnimation

def fourier_transform(tax,fax,f_t):
    M = outer(fax,tax)
    A = exp(1.j*M)
    return inner(A,f_t)

def search(value,axis,error):
    for it in arange(len(axis)):
        if abs(axis[it]-value) < error:
            return it
    print("Value not found.")  

Populating the interactive namespace from numpy and matplotlib


We generate the data from the .dat files: 

     t_pp_ax: the values of $t_{pp}$ corresponding to our initial sets of current data.
     
     E: array containing the probe field as function of $t_{gate}$.
     
     tax: the time axis (Method I) for the original sets of data and the probe field.
     
     J: these 60 arrays give the electric current corresponding to the $t_{pp}$  with their associated index in t_pp_ax.

In [5]:
probe_center = genfromtxt('probe_center.dat')
t_pp_ax = probe_center[:,1]
E = genfromtxt('E.dat')
tax = E[:1150,0]
trimmed_E = E[:1150,1]
for it in arange(1,60):
    globals()['J'+str(it)] = genfromtxt('J'+str(it)+'.dat')

Method I Calculation: We take the fourier transforms and calculate $\sigma$ for the horizontal cuts (Method I in the paper). We store this data in the array original_sigmas, which has dimensions 2x60x1150. The first array index identifies the real or imaginary part of the conductivity: the 0 index corresponds the real part and the 1 index to the imaginary part. The second index identifies which of the 60 sets of data we are referring to and is identified with t_pp_ax. The third index is identified with the frequency axis fax defined in the same cell. For example, [0,0,:] refers to the real part of the conductivity for data set J1. In order to smooth the fourier transform, we multiply the current and electric field by a decaying exponential and pad with zeros (see the definition of variable Jext in the function do_ft() below).

In [15]:
fax = fft.fftshift(fft.fftfreq(len(tax),d=0.2))
E_ext = E[:1150,1]* exp(-(tax-tax[0])/250)
E_w = fourier_transform(tax,fax,E_ext)

padding = zeros(575)
tau = 250.

def do_ft():
    original_sigmas = zeros((2,60,1150))
    for it in arange(1,60):
        J = globals()['J'+str(it)][:,1]
        Jext = concatenate((J,padding))*exp(-(tax-tax[0])/tau)
        original_sigmas[0,it-1,:] = real(fourier_transform(tax,fax,Jext) / E_w) 
        original_sigmas[1,it-1,:] = imag(fourier_transform(tax,fax,Jext) / E_w) 
    return original_sigmas

original_sigmas = do_ft()
savetxt("MethodIreal.dat",original_sigmas[0,:,:])
savetxt("MethodIimaginary.dat",original_sigmas[1,:,:])

Method II Calculation: Now we will do the calculation using Method II as described in the paper. We place all of the original data in a 2-dimensional array and interpolate along the vertical axis. Once we have plaed the interpolated data in an array, we shift these data to obtain the current as a function of $t_{gate}$ and $t_{pp}$ and take the fourier transform along the vertical axis (fixed $t_{gate}$) in this array, using these fourier transforms to calculate $\sigma$. As in Method I, in calculating the fourier transform we multiply the current and electric field by a decaying exponential and pad with zeros. The conductivitites are stored in the array vertical_sigmas, which has dimensions 2x1321x1150. As before, the first index indicates the real or imaginary part, the second index corresponds to the value of $t_{gate}$ (identified with indices of trimmedtax, defined in this cell), and the third index is identified with the same frequency axis fax defined above.

In [16]:
original_current = zeros((59,575))
for it in arange(59):
    original_current[it,:] = globals()['J'+str(it+1)][:,1]

yspan = max(t_pp_ax) - min(t_pp_ax)
new_t_pp_ax = linspace(min(t_pp_ax),max(t_pp_ax),int(5*yspan + 1))   # We define this to supercede t_pp_ax, since it is of smaller spacing.

interp_currents = zeros((1896,575))

for it in arange(575):
    strip = original_current[:,it]
    fit_func = interpolate.Akima1DInterpolator(t_pp_ax,strip)
    interp_currents[:,it] = fit_func(new_t_pp_ax)

shifted_current = zeros((1896,2470))

for it in arange(1896):
    dt = 0.2
    relative_time = new_t_pp_ax[it] - new_t_pp_ax[0]
    start_coord = int((1/dt)*relative_time)
    shifted_current[it,start_coord:start_coord+575] = interp_currents[it,:]
    
tmin = min(tax) + min(t_pp_ax)
tmax = max(tax) + max(t_pp_ax)

fulltax = linspace(tmin,tmax,3045)  # This time axis corresponds to all the values that t_gate may take on after our interpolation.

# Now we calculate the conductivities using Method II. We only calculate for a certain range of t_gate so that we are able to take complete 
# vertical strips.

for it in arange(575,1896):
    vert = flip(shifted_current[:,it])
    constrained = concatenate((vert[argmax(vert)-57:argmax(vert)+518],padding))*exp(-(tax-tax[0])/tau)
    FT = fourier_transform(tax,fax,constrained)
    globals()['sigma'+str(it)] = FT / E_w
    
trimmedtax = fulltax[575:1896]
trimmed_t_pp_ax = new_t_pp_ax[575:]
vertical_sigmas = zeros((2,1321,1150))
for it in arange(1321):
    vertical_sigmas[0,it,:] = real(globals()['sigma'+str(575+it)])
    vertical_sigmas[1,it,:] = imag(globals()['sigma'+str(575+it)])
    
savetxt("MethodIIreal.dat",vertical_sigmas[0,:,:])
savetxt("MethodIIimaginary.dat",vertical_sigmas[1,:,:])

(2, 1321, 1150)